In [ ]:
%pip install networkx

In [ ]:
import json
import networkx as nx
import numpy as np
from plotly import plot
import plotly.offline as pyo
# Set notebook mode to work in offline
pyo.init_notebook_mode()

# from utils import PlotlyJSONEncoder

In [ ]:
DG = nx.DiGraph()

In [ ]:
DG.clear()

In [ ]:
f = open('test-fc-loop.json')

In [ ]:
!pwd

In [ ]:
fc = json.loads(f.read())

In [ ]:
fc

In [ ]:
elems = fc['nodes']

In [ ]:
for el in elems:
    if 'id' in el:
        print(el['id'].split('-')[0])

## Add nodes

In [ ]:
for i in range(len(elems)):
    el = elems[i]
    if 'source' not in el:
        DG.add_node(i+1, pos=(el['position']['x'], el['position']['y']),id=el['id'])
        elems[i]['index'] = i+1
        elems[i]['label'] = el['id'].split('-')[0]

In [ ]:
pos = nx.get_node_attributes(DG,'pos')

In [ ]:
pos

In [ ]:
nx.draw(DG, pos, with_labels=True)

## Add edges

In [ ]:
edge_label_dict = {}
def get_tuple(edge):
    e = [-1, -1]
    src_id = edge['source']
    tgt_id = edge['target']

    if tgt_id not in edge_label_dict.keys():
            edge_label_dict[tgt_id] = []

    edge_label_dict[tgt_id].append({
        'source': src_id,
        'label': edge['label'] if 'label' in edge else "default",
        'sourceHandle': edge['sourceHandle'],
        'targetHandle': edge['targetHandle']
    })

    # iterate through all nodes looking for matching edge
    for el in elems:
        if 'id' in el:
            if el['id'] == src_id:
                e[0] = el['index']
            elif el['id'] == tgt_id:
                e[1] = el['index']
    return tuple(e)
edges = fc['edges']
for i in range(len(edges)):
    el = edges[i]

    # element is an edge
    e = get_tuple(el)
    DG.add_edge(*e)

In [ ]:
DG.number_of_edges()

In [ ]:
list(DG.nodes)

In [ ]:
edges = list(DG.edges)
print(edges)
# for edge in edges:
#     print(edge[1])

In [ ]:
[{el['id']: el['type']} for el in elems]

In [ ]:
nx.draw(DG, pos, with_labels=True)

## Add labels to DG nodes

In [ ]:
labels = {}

for el in elems:
    # if element is not a node
    if 'source' not in el:
        labels[el['index']] = el['data']['func']
                
nx.set_node_attributes(DG, labels, 'cmd')
nx.draw(DG, pos, with_labels=True, labels = labels)

In [ ]:
# nodes_by_id = dict()
# for n, nd in DG.nodes().items():
#     if n is not None:
#         nodes_by_id[n] = nd
# nodes_by_id

In [ ]:
DG

In [ ]:
list(nx.dfs_tree(DG, source=1))

In [ ]:
for n, nd in DG.nodes().items():
    print('node', n, 'node data', nd)

In [ ]:
DG.edges

In [ ]:
def get_node_data_by_id():
    nodes_by_id = dict()
    for n, nd in DG.nodes().items():
        nodes_by_id[n] = nd
    return nodes_by_id

In [ ]:
for n in nx.topological_sort(DG):
    print(n, end=' ')

In [ ]:
edge_label_dict

#### DFS Algorithms

In [ ]:
class Graph:
    def __init__(self,DG,edge_label_dict):
        self.DG = DG
        self.edges = DG.edges
        self.nodes = DG.nodes
        self.edge_label_dict = edge_label_dict
        self.adjList = {}
        self.make_adjancency_list()

    def get_node_data_by_id(self):
        nodes_by_id = dict()
        for n, nd in self.DG.nodes().items():
            nodes_by_id[n] = nd
        return nodes_by_id

    def make_adjancency_list(self):
        for (src,dest) in self.edges:

            if src not in self.adjList.keys():
                self.adjList[src] = []

            for value in self.edge_label_dict[self.get_node_data_by_id()[dest]['id']]:
                
                if value['source'] == self.get_node_data_by_id()[src]['id']:
                    sourceHandle = value['sourceHandle']

            self.adjList[src].append({
                'target_node_id': self.get_node_data_by_id()[dest]['id'],
                'src_node_id':self.get_node_data_by_id()[src]['id'],
                'target_node':dest,
                'handle':sourceHandle
            })

In [ ]:
get_node_data_by_id()[4]

In [ ]:
def DFS(graph,source,discovered,current_loop_nodes,hashmap):

    cmd = get_node_data_by_id()[source]['cmd']
    id = get_node_data_by_id()[source]['id']

    # print(cmd)

    if source not in graph.adjList.keys():
        hashmap[id] = current_loop_nodes.copy()
        discovered[source-1] = True
        return

    body = []
    end = []

    # checking if the source is LOOP type
    if cmd == 'LOOP':

        current_loop_nodes.append(id)

        # find the end & body source Handle
        for value in graph.adjList[source]:
            if value['handle'] == 'body':
                body.append(value['target_node'])
            if value['handle'] == 'end':
                end.append(value['target_node'])
    
        # traversing the body node first
        for value in body:
            if not discovered[value-1]:
                DFS(graph=graph,source=value,discovered=discovered,current_loop_nodes=current_loop_nodes,hashmap=hashmap)

        current_loop_nodes.pop()
        hashmap[id] = current_loop_nodes.copy()

        for value in end:
            if not discovered[value-1]:
                DFS(graph=graph,source=value,discovered=discovered,current_loop_nodes=current_loop_nodes,hashmap=hashmap)
        
        discovered[source-1] = True
    else:
        for value in graph.adjList[source]:
            if not discovered[value['target_node']-1]:
                DFS(graph=graph,source=value['target_node'],discovered=discovered,current_loop_nodes=current_loop_nodes,hashmap=hashmap)
        hashmap[id] = current_loop_nodes.copy()        
        discovered[source-1] = True

In [ ]:
graph = Graph(DG,edge_label_dict)
discovered = [False] * len(list(DG.nodes))

# finding the source of dfs tree
dfs_source = []
for node in DG.nodes:
    if len(list(DG.predecessors(node))) == 0:
        dfs_source.append(node)

hash_map = {}
current_loop_nodes = []
DFS(graph=graph,source=dfs_source[0],discovered=discovered,current_loop_nodes=current_loop_nodes,hashmap = hash_map)

hash_map

In [ ]:
hash_map_loop = {}
for key,value in hash_map.items():
    if len(value) > 0:
        for loop_id in value:
            if loop_id not in hash_map_loop.keys():
                hash_map_loop[loop_id] = []
            hash_map_loop[loop_id].append(key)
hash_map_loop

In [ ]:
topological_sorting_list = [get_node_data_by_id()[node]['id'] for node in list(nx.topological_sort(DG))]
topological_sorting_list

In [ ]:
sorting_order_loopnodes = {}

for key,nodes in hash_map_loop.items():
    sorting_order = [topological_sorting_list.index(node) for node in nodes]
    sorting_order.sort()
    sorting_order_loopnodes[key] = [topological_sorting_list[node_id] for node_id in sorting_order]
    
sorting_order_loopnodes

In [ ]:
# removing dependant loop nodes
for key,parents in hash_map.items():
    if 'LOOP' in key and len(parents) > 0:
        for parent in parents:

            parent_loop_nodes = sorting_order_loopnodes[parent]
            child_loop_nodes = sorting_order_loopnodes[key]

            print('parent loop node: ',parent_loop_nodes)
            print('child loop node: ',child_loop_nodes)

            for node in child_loop_nodes:
                parent_loop_nodes.remove(node) if node in parent_loop_nodes else ''

            sorting_order_loopnodes[parent] = parent_loop_nodes
sorting_order_loopnodes

# REDIS & IMPORTS

In [ ]:
from redis import Redis
from rq import Queue
from rq.job import Job

In [ ]:
Redis()

In [ ]:
conn = Redis()
conn

In [ ]:
q = Queue(connection=conn)
q

In [ ]:
from GENERATORS import *
from TRANSFORMERS import *
from VISORS import *

In [ ]:
func = getattr(globals()['LINSPACE'], 'LINSPACE')
func

In [ ]:
q.enqueue(func)

In [ ]:
q

In [ ]:
def report_success(job, connection, result, *args, **kwargs):
    print('success', result)

def report_failure(job, connection, type, value, traceback):
    print('failure')
    print(job, connection, type, value, traceback)

In [ ]:
topological_sorting = nx.topological_sort(DG)
nodes_by_id = get_node_data_by_id()

for n in topological_sorting:

    cmd = nodes_by_id[n]['cmd']
    ctrls = dict()

    if cmd.replace('.','',1).isdigit():
        ctrls['constant'] = cmd
        cmd = 'CONSTANT'   
    
    func = getattr(globals()[cmd], cmd)
    job_id = 'job_id_{0}'.format(n)

    print('>>> visiting node *** {0} *** ({1})'.format(cmd, n))
    print('Queueing function... ...', func)
    
    node_predecessors = DG.predecessors(n)
    
    if len(list(node_predecessors)) == 0:
        print ('{0} ({1}) has no predecessors'.format(cmd, n))
        q.enqueue(func, job_id = job_id, kwargs={'ctrls': ctrls})
    else:
        previous_job_ids = []
        for p in DG.predecessors(n):
            prev_cmd = DG.nodes[p]['cmd']
            prev_job_id = 'job_id_{0}'.format(p)
            previous_job_ids.append(prev_job_id)
            print(prev_cmd, 'is a predecessor to', cmd)
        q.enqueue(func,
            job_id = job_id,
            kwargs={'ctrls': ctrls, 'previous_job_ids': previous_job_ids},            
            depends_on = previous_job_ids)

In [ ]:
nodes_by_id

# Get end nodes

In [ ]:
end_nodes = [x for x in DG.nodes() if DG.out_degree(x)==0 and DG.in_degree(x)==1]

In [ ]:
end_nodes

In [ ]:
end_node_results = []

for n in end_nodes:
    job_id = 'job_id_{0}'.format(n)
    nd = get_node_data_by_id()[n]
    print(nd)
    job = Job.fetch(job_id, connection=Redis())
    payload = job.result
    print(str(payload['data'][0]['type']))
    end_node_results.append({nd['cmd']: payload})
    
end_node_results

In [ ]:
!pwd

In [ ]:
f = open("results.json", "w")
f.write(json.dumps(end_node_results, cls=PlotlyJSONEncoder))
f.close()

# Import all functions in folder module

In [ ]:
import os
import sys

In [ ]:
sys.version

In [ ]:
os.chdir('/Users/jackparmer/Desktop/projects/daq-labs/PYTHON/WATCH')

In [ ]:
os.getcwd()

## Change to FUNCTIONS folder

In [ ]:
os.chdir('../FUNCTIONS')

In [ ]:
os.getcwd()

In [ ]:
from GENERATORS import *
from TRANSFORMERS import *
from VISORS import *

In [ ]:
SCATTER

In [ ]:
SINE.SINE

In [ ]:
import inspect
code =inspect.getsource(getattr(globals()['SCATTER'], 'SCATTER'));
code

In [ ]:
import os
os.listdir('../FUNCTIONS/SIMULATIONS')

In [ ]:
globals()['GENERATORS'] = __import__('GENERATORS')

In [ ]:
SINE